# Pendulum Period vs. Initial Angle

The initial boilerplate to start a sicmutils investigation:

In [1]:
(require '[clojupyter.misc.helper :as helper])
(run! helper/add-dependencies '[[net.littleredcomputer/sicmutils "0.11.0-SNAPSHOT"]
                                [thi.ng/geom "0.0.908"]])
(ns double-pendulum
    (:refer-clojure :exclude [partial zero? + - * / ref])
    (:require [sicmutils.env :refer :all]
              [sicmutils.examples.pendulum :as pendulum]
              [clojupyter.misc.display :as display]
              [thi.ng.geom.viz.core :as viz]
              [thi.ng.geom.svg.core :as svg]
              [thi.ng.color.core :as color]))

The pendulum library in the examples directory contains the Lagrangian for the simple pendulum with moving pivot. For this investigation, we want a pendulum with a fixed pivot, so we supply a constant function for the motion of the pivot. Let's double-check that the Lagrangian for the pendulum agrees with what we find on Wikipedia.

In [2]:
  ;; The coordinates of the pivot as a function of time
(defn fixed [t] (up 0 0))
  ;; Obtain a fixed-pivot Lagrangian from the moving-pivot Lagrangian
(defn L [m l g] (pendulum/L m l g fixed))
  ;; Bind the contants to symbolic values, and supply symbolic initial data
(tex$ ((L 'm 'l 'g) (up 't 'theta 'thetadot)))  

That looks [correct](https://en.wikipedia.org/wiki/Lagrangian_mechanics#Pendulum_on_a_movable_support). Now we form the state derivative:

In [3]:
(defn state-derivative [m l g]
    (Lagrangian->state-derivative (L m l g)))
(tex$$ ((state-derivative 'm 'l 'g) (up 't 'theta 'thetadot)))

So far so good. The next step is to integrate the motion for various choices of initial displacement. We'll set $m = l = 1, g = 9.8$ here, and call the initial angle $\theta_0$. We will run the simulation for 4 seconds, sampling every 1/60 second. Following that is some plotting boilerplate. We resume commentary at the graph below.

In [4]:
(defn p [theta_0]
    (integrate-state-derivative state-derivative [1 1 9.8] (up 0 theta_0 0) 4 1/60))

#'double-pendulum/p

In [5]:
(def spec
  {:x-axis (viz/linear-axis
            {:domain [0 4]
             :range  [50 (- 985 10)]
             :major 1
             :pos    550})
   :y-axis (viz/linear-axis
            {:domain      [(- Math/PI) Math/PI]
             :range       [550 20]
             :major       1
             :minor       0.2
             :pos         50
             :label-dist  15
             :label-style {:text-anchor "end"}})
   :grid   {:attribs {:stroke "#caa"}
            :minor-x true
            :minor-y false}
   :data   (for [a (range 0.15 2 0.15)]
               {:values  (p a)
                :attribs {:stroke (color/as-css (color/rgba 0.8 (/ a 2) 0 1)) :stroke-width "2pt"}
                :layout  viz/svg-line-plot})
})

#'double-pendulum/spec

Now let's look at the output. For a small displacement, the high-school (linearized) calculation shows we should expect a period of $2\pi\sqrt{\frac{l}g}\approx 2.0$ in our case with $l=1$ and $g=9.8$. As the initial angular displacement increases, however, the period begins to rise.

In [6]:
(->> spec
     viz/svg-plot2d-cartesian
     (svg/svg {:width 985 :height 600})
     svg/serialize
     display/html)

<?xml version="1.0"?>
 <polyline fill="none" points="50.00,272.35 53.85,272.36 57.71,272.42 61.56,272.50 65.42,272.62 69.27,272.77 73.13,272.96 76.98,273.18 80.83,273.43 84.69,273.71 88.54,274.02 92.40,274.37 96.25,274.74 100.10,275.14 103.96,275.56 107.81,276.02 111.67,276.49 115.52,276.99 119.38,277.51 123.23,278.05 127.08,278.61 130.94,279.19 134.79,279.79 138.65,280.39 142.50,281.01 146.35,281.64 150.21,282.29 154.06,282.93 157.92,283.59 161.77,284.24 165.63,284.90 169.48,285.56 173.33,286.22 177.19,286.87 181.04,287.52 184.90,288.17 188.75,288.80 192.60,289.42 196.46,290.03 200.31,290.63 204.17,291.22 208.02,291.78 211.88,292.33 215.73,292.86 219.58,293.36 223.44,293.84 227.29,294.30 231.15,294.74 235.00,295.15 238.85,295.52 242.71,295.88 246.56,296.20 250.42,296.49 254.27,296.75 258.13,296.98 261.98,297.17 265.83,297.34 269.69,297.47 273.54,297.56 277.40,297.62 281.25,297.65 285.10,297.64 288.96,297.60 292.81,297.53 296.67,297.42 300.52,297.28 304.38,297.10 308.23,296.89 312.08,296.65 315.94,296.38 319.79,296.07 323.65,295.74 327.50,295.37 331.35,294.98 335.21,294.56 339.06,294.12 342.92,293.65 346.77,293.16 350.63,292.64 354.48,292.11 358.33,291.55 362.19,290.98 366.04,290.39 369.90,289.79 373.75,289.17 377.60,288.54 381.46,287.91 385.31,287.26 389.17,286.61 393.02,285.95 396.88,285.29 400.73,284.63 404.58,283.98 408.44,283.32 412.29,282.67 416.15,282.02 420.00,281.39 423.85,280.76 427.71,280.15 431.56,279.54 435.42,278.96 439.27,278.39 443.13,277.83 446.98,277.30 450.83,276.79 454.69,276.30 458.54,275.83 462.40,275.39 466.25,274.97 470.10,274.59 473.96,274.23 477.81,273.89 481.67,273.59 485.52,273.32 489.38,273.09 493.23,272.88 497.08,272.71 500.94,272.57 504.79,272.46 508.65,272.39 512.50,272.35 516.35,272.35 520.21,272.38 524.06,272.45 527.92,272.55 531.77,272.68 535.63,272.85 539.48,273.04 543.33,273.28 547.19,273.54 551.04,273.83 554.90,274.16 558.75,274.51 562.60,274.90 566.46,275.31 570.31,275.74 574.17,276.21 578.02,276.69 581.88,277.20 585.73,277.73 589.58,278.28 593.44,278.85 597.29,279.43 601.15,280.03 605.00,280.64 608.85,281.27 612.71,281.90 616.56,282.55 620.42,283.20 624.27,283.85 628.13,284.51 631.98,285.17 635.83,285.83 639.69,286.48 643.54,287.14 647.40,287.78 651.25,288.42 655.10,289.05 658.96,289.67 662.81,290.28 666.67,290.87 670.52,291.45 674.38,292.00 678.23,292.54 682.08,293.06 685.94,293.56 689.79,294.03 693.65,294.48 697.50,294.91 701.35,295.30 705.21,295.67 709.06,296.01 712.92,296.32 716.77,296.60 720.63,296.85 724.48,297.06 728.33,297.24 732.19,297.39 736.04,297.51 739.90,297.59 743.75,297.64 747.60,297.65 751.46,297.63 755.31,297.58 759.17,297.49 763.02,297.36 766.88,297.21 770.73,297.02 774.58,296.80 778.44,296.54 782.29,296.26 786.15,295.94 790.00,295.59 793.85,295.22 797.71,294.82 801.56,294.39 805.42,293.93 809.27,293.45 813.13,292.95 816.98,292.43 820.83,291.89 824.69,291.32 828.54,290.74 832.40,290.15 836.25,289.54 840.10,288.92 843.96,288.29 847.81,287.65 851.67,287.00 855.52,286.34 859.38,285.69 863.23,285.03 867.08,284.37 870.94,283.71 874.79,283.06 878.65,282.41 882.50,281.77 886.35,281.13 890.21,280.51 894.06,279.90 897.92,279.30 901.77,278.72 905.63,278.16 909.48,277.61 913.33,277.09 917.19,276.59 921.04,276.11 924.90,275.65 928.75,275.22 932.60,274.81 936.46,274.44 940.31,274.09 944.17,273.77 948.02,273.48 951.88,273.22 955.73,273.00 959.58,272.81 963.44,272.65 967.29,272.52 971.15,272.43" stroke-width="2pt" stroke="#cc1300" /><polyline fill="none" points="50.00,259.69 53.85,259.73 57.71,259.83 61.56,260.00 65.42,260.24 69.27,260.54 73.13,260.91 76.98,261.34 80.83,261.84 84.69,262.39 88.54,263.01 92.40,263.69 96.25,264.43 100.10,265.22 103.96,266.07 107.81,266.96 111.67,267.91 115.52,268.90 119.38,269.93 123.23,271.01 127.08,272.12 130.94,273.27 134.79,274.45 138.65,275.66 142.50,276.89 146.35,278.15 150.21,279.42 154.06,280.71 157.92,282.01 161.77,283.32 165.63,284.64 169.48,285.95 173.33,287.26 177.19,288.57 181.04,289.87 184.90,291.15 188.75,292.42 192.60,293.66 196.46,294.8

In [7]:
(tex$ ((Lagrangian->energy (L 'm 'l 'g)) (up 't 'theta 'thetadot)))

The energy of the system is conserved. Assuming we raise the bob to an angle $\theta_0$ and release it, the total energy must remain equal to the potential energy initially present:

In [8]:
(tex$ ((pendulum/V 'm 'l 'g fixed) (up 't 'theta_0 'thetadot_0)))

Equating these expressions allows us to write $\dot\theta$ in terms of $\theta$ and $\theta_0$. We can then form the integral for the period 
$$\mathrm{period}(\theta_0)=4\int_0^{\theta_0}\frac{1}{\dot\theta}d\theta$$
We find that
$$\mathrm{period}(\theta_0)=\frac{4}{\sqrt{2g}}\int_0^{\theta_0}\frac{1}{\sqrt{\cos\theta - \cos\theta_0}}d\theta$$


In [17]:
(defn integrand [g theta0]
    (fn [theta] (/ (sqrt (* 2 g (- (cos theta) (cos theta0)))))))
(defn period [theta0]
    (* 4 (definite-integral (integrand 9.8 theta0) 0 (- theta0 0.00001) )))

#'double-pendulum/period

We interrupt this broadcast to inform the viewers that the integrator we are currently using is not up to the task of computing this improper integral (the integrand blows up at the right-hand edge of the interval).